In [ ]:
from google.colab import drive

# ✅ Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#first prompt

import openai
import pandas as pd
import csv
import json
import os
import time
from google.colab import drive

# ✅ Mount Google Drive
drive.mount('/content/drive')

# ✅ OpenAI API Key (from environment variable)
client = openai.OpenAI(api_key="")

# ✅ File paths
input_csv = "PuppetScripts_V2.csv"
output_csv = "chatgpt3.5_cwe_puppet_3.5.csv"

# ✅ Load input CSV
df = pd.read_csv(input_csv)

# ✅ CWE-only prompt
def create_prompt(code_snippet):
    return f"""
        You are a security expert specializing in Puppet configuration security.
        Your task is to analyze the following Puppet code and identify only critical security misconfigurations issues that could lead to vulnerabilities and have a real CWE ID.
        Focus only on the available code and avoid making assumptions about unavailable details
        Puppet Script to Analyze:
        ```puppet
        {code_snippet}
        ```

        Response Format:
        Return the response in **valid JSON format**:

        - If there are critical issues provide unique misconfigured code:
        ```json
        {{
          "issues_found": [
            {{
              "misconfigured_snippet": "<actual misconfigured code snippet>",
              "related_cwe": "<CWE-ID>"
            }}
          ],
          "misconfiguration_label": 1
        }}
        ```

        - If there are NO critical issues:
        ```json
        {{
          "issues_found": [],
          "misconfiguration_label": 0
        }}
        ```

        Instructions:
        - Only list real misconfigurations tied to known CWE IDs.
        - **Do NOT** insert `"N/A"` entries into `issues_found`.
        - Do not include syntax errors.
        - If no critical misconfiguration is found, the `issues_found` array must be completely empty `[]`.
        - If no critical misconfiguration is found, set `"misconfiguration_label": 0`.
    """

# ✅ Query OpenAI and safely parse JSON
def send_to_openai(prompt, row_index):
    while True:
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=4000,
                temperature=0.2,
                top_p=0.3,
            )
            response_text = response.choices[0].message.content.strip()

            try:
                start_index = response_text.index('{')
                end_index = response_text.rindex('}') + 1
                json_part = response_text[start_index:end_index]
                response_json = json.loads(json_part)
                print(f"\n🔹 GPT Response (Row {row_index + 1}): {response_text[:120]}...")
            except (ValueError, json.JSONDecodeError) as e:
                print(f"❌ JSON Error Row {row_index + 1}: {e}")
                return ["JSON_PARSE_ERROR"], 1

            issues_list = response_json.get("issues_found", [])
            label = response_json.get("misconfiguration_label", 0)

            # Extract CWEs
            cwes = [issue.get("related_cwe", "").strip() for issue in issues_list if "related_cwe" in issue]

            return list(set(cwes)), label  # unique CWE list

        except Exception as e:
            print(f"❌ API Error Row {row_index + 1}: {e}")
            time.sleep(2)

# ✅ Initialize output file
if not os.path.exists(output_csv):
    with open(output_csv, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Script.Content", "Defect.Label", "Combined.CWEs", "LLM.Label"])

# ✅ Process all rows
for index, row in df.iterrows():
    script = str(row.get("Script.Content", "")).strip()
    label = str(row.get("Defect.Label", "")).strip()

    if not script:
        print(f"⚠️ Skipping Row {index + 1} (Empty Script)")
        continue

    try:
        time.sleep(0.2)
        prompt = create_prompt(script)
        cwes, misconf_label = send_to_openai(prompt, index)

        cwe_str = "; ".join(cwes) if cwes else ""


        with open(output_csv, mode="a", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow([script, label, cwe_str, misconf_label])

    except Exception as e:
        print(f"❌ Unexpected error Row {index + 1}: {e}")

print("\n✅ All rows processed. Single-row CWE output complete.")


Streaming output truncated to the last 5000 lines.

🔹 GPT Response (Row 1188): ```json
{
  "issues_found": [
    {
      "misconfigured_snippet": "file_line { 'root_helper_daemon': line => 'neutron a...

🔹 GPT Response (Row 1189): ```json
{
  "issues_found": [
    {
      "misconfigured_snippet": "$deploy_dir = \"/var/lib/puppetmaster/deploy\"",
   ...

🔹 GPT Response (Row 1190): ```json
{
  "issues_found": [
    {
      "misconfigured_snippet": "postgresql_password($user, $password)",
      "relat...

🔹 GPT Response (Row 1191): ```json
{
  "issues_found": [
    {
      "misconfigured_snippet": "class toplevel::server::buildmaster::mozilla inherit...

🔹 GPT Response (Row 1192): ```json
{
  "issues_found": [
    {
      "misconfigured_snippet": "file { '/etc/rc.local': ensure => file, mode => '077...

🔹 GPT Response (Row 1193): ```json
{
  "issues_found": [
    {
      "misconfigured_snippet": "mode => 0777",
      "related_cwe": "CWE-732"
    }
...

🔹 GPT Response (Row 1194): ```json
{

In [ ]:
#main prompt
import openai
import pandas as pd
import csv
import json
import os
import time
from google.colab import drive

# ✅ Mount Google Drive
drive.mount('/content/drive')

# ✅ OpenAI API Key (from environment variable)
client = openai.OpenAI(api_key="")

# ✅ File paths
input_csv = "/content/drive/My Drive/expermintstarcoder/PuppetScripts_V2.csv"
output_csv = "/content/drive/My Drive/expermintstarcoder/imp/chatgpt3.5_cwe_puppet_3.5_second.csv"

# ✅ Load input CSV
df = pd.read_csv(input_csv)

# ✅ CWE-only prompt
def create_prompt(code_snippet):
    return f"""
        You are a security expert specializing in Puppet configuration security.
        Your task is to analyze the following Puppet code and identify only critical security misconfigurations issues that could lead to vulnerabilities and have a real CWE ID.
        Focus only on the available code and avoid making assumptions about unavailable details
        Puppet Script to Analyze:
        ```puppet
        {code_snippet}
        ```

        Response Format:
        Return the response in **valid JSON format**:

        - If there are critical issues provide unique misconfigured code:
        ```json
        {{
          "issues_found": [
            {{
              "misconfigured_snippet": "<actual misconfigured code snippet>",
              "related_cwe": "<CWE-ID>"
            }}
          ],
          "misconfiguration_label": 1
        }}
        ```

        - If there are NO critical issues:
        ```json
        {{
          "issues_found": [],
          "misconfiguration_label": 0
        }}
        ```

        Instructions:
        - Only list real misconfigurations tied to known CWE IDs.
        - **Do NOT** insert `"N/A"` entries into `issues_found`.
        - Do not include syntax errors.
        - If no critical misconfiguration is found, the `issues_found` array must be completely empty `[]`.
        - If no critical misconfiguration is found, set `"misconfiguration_label": 0`.
    """


def send_to_openai(prompt, row_index):
    while True:
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=4000,
                temperature=0.2,
                top_p=0.3,
            )
            response_text = response.choices[0].message.content.strip()

            try:
                start_index = response_text.index('{')
                end_index = response_text.rindex('}') + 1
                json_part = response_text[start_index:end_index]
                response_json = json.loads(json_part)
                print(f"\n🔹 GPT Response (Row {row_index + 1}): {response_text[:120]}...")
            except (ValueError, json.JSONDecodeError) as e:
                print(f"❌ JSON Error Row {row_index + 1}: {e}")
                return ["JSON_PARSE_ERROR"], 1, None  # Return None for misconfigured_snippet if error occurs

            issues_list = response_json.get("issues_found", [])
            label = response_json.get("misconfiguration_label", 0)
            print(f"\n🔹 GPT Response (Row {row_index + 1}): {response_text[:120]}...")
            # Extract CWEs and misconfigured snippets
            cwes = [issue.get("related_cwe", "").strip() for issue in issues_list if "related_cwe" in issue]
            snippets = [issue.get("misconfigured_snippet", "").strip() for issue in issues_list if "misconfigured_snippet" in issue]

            misconfigured_snippet = snippets[0] if snippets else None  # If there are snippets, pick the first one

            return list(set(cwes)), label, misconfigured_snippet  # Return the snippet along with CWEs and label

        except Exception as e:
            print(f"❌ API Error Row {row_index + 1}: {e}")
            time.sleep(2)

# ✅ Initialize output file
if not os.path.exists(output_csv):
    with open(output_csv, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Script.Content", "Defect.Label", "Combined.CWEs", "Misconfigured.Snippet", "LLM.Label"])

# ✅ Process all rows
for index, row in df.iterrows():
    script = str(row.get("Script.Content", "")).strip()
    label = str(row.get("Defect.Label", "")).strip()

    if not script:
        print(f"⚠️ Skipping Row {index + 1} (Empty Script)")
        continue

    try:
        time.sleep(0.2)
        prompt = create_prompt(script)
        cwes, misconf_label, misconfigured_snippet = send_to_openai(prompt, index)

        cwe_str = "; ".join(cwes) if cwes else ""
        misconfigured_snippet_str = misconfigured_snippet if misconfigured_snippet else ""

        # Now save all the necessary information including the snippet
        with open(output_csv, mode="a", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow([script, label, cwe_str, misconfigured_snippet_str, misconf_label])

    except Exception as e:
        print(f"❌ Unexpected error Row {index + 1}: {e}")

print("\n✅ All rows processed. Single-row CWE output complete.")

Streaming output truncated to the last 5000 lines.
{
  "issues_found": [
    {
      "misconfigured_snippet": "ensure => ensure_link($ensure)",
      "related_cwe"...

🔹 GPT Response (Row 1572): ```json
{
  "issues_found": [
    {
      "misconfigured_snippet": "$log_dir = '/var/log/rally'",
      "related_cwe": "...

🔹 GPT Response (Row 1572): ```json
{
  "issues_found": [
    {
      "misconfigured_snippet": "$log_dir = '/var/log/rally'",
      "related_cwe": "...

🔹 GPT Response (Row 1573): ```json
{
  "issues_found": [
    {
      "misconfigured_snippet": "$database_connection = 'sqlite:///var/lib/glance/gla...

🔹 GPT Response (Row 1573): ```json
{
  "issues_found": [
    {
      "misconfigured_snippet": "$database_connection = 'sqlite:///var/lib/glance/gla...

🔹 GPT Response (Row 1574): ```json
{
  "issues_found": [
    {
      "misconfigured_snippet": "$sahara_password = pick($sahara_hash['user_password'...

🔹 GPT Response (Row 1574): ```json
{
  "issues_found": [
    {
      "mi

In [ ]:
import pandas as pd
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    roc_auc_score,
    precision_recall_curve,
    auc
)

# Load dataset
df = pd.read_csv("chatgpt3.5_cwes_puppet.csv")

# Clean data
df = df.dropna(subset=['verification', 'LLM.Label'])  # Drop rows with missing values
df['verification'] = pd.to_numeric(df['verification'], errors='coerce')
df['LLM.Label'] = pd.to_numeric(df['LLM.Label'], errors='coerce')
df = df.dropna(subset=['verification', 'LLM.Label'])  # Drop again in case conversion made NaNs

# Set truth and prediction
true_labels = df['verification']        # ✅ Human verified truth
predicted_labels = df['LLM.Label']      # ✅ Model output

# Compute classification metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision, recall, f1, _ = precision_recall_fscore_support(
    true_labels,
    predicted_labels,
    average='binary',
    pos_label=1,        # 1 means "misconfiguration found"
    zero_division=0
)

# ROC AUC (only valid if both 0 and 1 exist in truth labels)
try:
    roc_auc = roc_auc_score(true_labels, predicted_labels)
except ValueError:
    roc_auc = None

# PR AUC
precision_vals, recall_vals, _ = precision_recall_curve(true_labels, predicted_labels)
pr_auc = auc(recall_vals, precision_vals)

# Print results
print(f"Accuracy             : {accuracy:.4f}")
print(f"Precision            : {precision:.4f}")
print(f"Recall               : {recall:.4f}")
print(f"F1-Score             : {f1:.4f}")
if roc_auc is not None:
    print(f"ROC AUC              : {roc_auc:.4f}")
else:
    print("ROC AUC              : Undefined (only one class in true labels)")
print(f"Precision-Recall AUC : {pr_auc:.4f}")


Accuracy             : 0.7661
Precision            : 0.7652
Recall               : 0.9980
F1-Score             : 0.8662
ROC AUC              : 0.5170
Precision-Recall AUC : 0.8824
